Создаём submition файл и сохраняем результаты деткции в папку `detected_books`. **Также** **НЕОБХОДИМО** поместить папку из тестового задания `task_images` в `data/`

In [ ]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO
import numpy as np

# Загрузка обученной модели
model = YOLO("models/path/to/model.pt")  # укажите путь к вашей модели

test_images_path = 'data/task_images'

output_dir = 'detected_books'
os.makedirs(output_dir, exist_ok=True)

results = []
def calculate_iou(box1, box2):
    x1_i = max(box1[0], box2[0])
    y1_i = max(box1[1], box2[1])
    x2_i = min(box1[2], box2[2])
    y2_i = min(box1[3], box2[3])
    
    intersection = max(0, x2_i - x1_i) * max(0, y2_i - y1_i)
    
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

for image_name in os.listdir(test_images_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_images_path, image_name)
        image = cv2.imread(image_path)
        
        # Предсказание с настройками NMS
        predictions = model(image_path, iou=0.7, conf=0.75)  # Настройте параметры
        
        # Подсчет уникальных детекций
        num_books = 0
        for prediction in predictions:
            # Получаем bounding boxes
            boxes = prediction.boxes
            num_books = len(boxes)
            
            # Применяем дополнительную логику для фильтрации дубликатов
            if len(boxes) > 1:
                # Фильтруем сильно перекрывающиеся боксы
                filtered_boxes = []
                for i, box1 in enumerate(boxes):
                    keep = True
                    for j, box2 in enumerate(boxes):
                        if i != j and calculate_iou(box1.xyxy[0].cpu().numpy(), 
                                                   box2.xyxy[0].cpu().numpy()) > 0.5:
                            if box1.conf[0] < box2.conf[0]:
                                keep = False
                                break
                    if keep:
                        filtered_boxes.append(box1)
                
                num_books = len(filtered_boxes)
                boxes = filtered_boxes
            
            # Визуализация
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                confidence = box.conf[0].cpu().numpy()
                
                cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                label = f"Book: {confidence:.2f}"
                cv2.putText(image, label, (int(x1), int(y1)-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Сохраняем изображение
        output_path = os.path.join(output_dir, f"nms_{image_name}")
        cv2.imwrite(output_path, image)
        
        # Добавляем в результаты
        image_id = os.path.splitext(image_name)[0]
        results.append({'image_id': image_id, 'number_of_books': num_books})

